In [1]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())


2.3.1
CUDA available: True


Name	        Example Value	Data Type	Description
settings_version'0.0.4'	            str	    Ultralytics settings version 
datasets_dir	'/path/to/datasets'	str	 The directory where the datasets are stored
weights_dir  	'/path/to/weights'	str	 The directory where the model weights are stored
runs_dir	    '/path/to/runs'	    str	 The directory where the experiment runs are stored
uuid	        'a1b2c3d4'	        str	 The unique identifier for the current settings
sync	        True	            bool	Whether to sync analytics and crashes to HUB
api_key	        ''	                str	    Ultralytics HUB API Key
clearml	        True	            bool	Whether to use ClearML logging
comet	        True	            bool	Whether to use Comet ML for experiment tracking and visualization
dvc	            True	            bool	Whether to use DVC for experiment tracking and version control
hub	            True	            bool	Whether to use Ultralytics HUB integration
mlflow	        True	            bool	Whether to use MLFlow for experiment tracking
neptune	        True	            bool	Whether to use Neptune for experiment tracking
raytune	        True	            bool	Whether to use Ray Tune for hyperparameter tuning
tensorboard	    True	            bool	Whether to use TensorBoard for visualization
wandb	        True	            bool	Whether to use Weights & Biases logging
vscode_msg	    True	            bool	When VS Code terminal detected, enables prompt to download Ultralytics-Snippets extension.

In [5]:
import os
import random
import shutil
import yaml

# 設定隨機種子，確保分割可重現
random.seed(42)

# 資料分割比例
split_ratios = {
    "train": 0.8,
    "val": 0.1,
    "test": 0.1
}

# 資料集路徑
dataset_path = "dataset"

# 分割後的目標資料夾名稱
output_dirs = {
    "train": os.path.join(dataset_path, "train"),
    "val": os.path.join(dataset_path, "val"),
    "test": os.path.join(dataset_path, "test")
}

# 創建分割後的資料夾
def create_dirs():
    for split_dir in output_dirs.values():
        os.makedirs(split_dir, exist_ok=True)

# 分割資料
def split_dataset():
    class_names = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    for class_name in class_names:
        class_path = os.path.join(dataset_path, class_name)
        
        # 檢查是否為有效類別資料夾
        if not os.path.isdir(class_path) or class_name in output_dirs:
            continue
        
        # 取得該類別下所有圖片檔案
        images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)
        
        # 計算每個分割集的數量
        total_images = len(images)
        train_end = int(total_images * split_ratios["train"])
        val_end = train_end + int(total_images * split_ratios["val"])
        
        # 分割成 train, val, test
        splits = {
            "train": images[:train_end],
            "val": images[train_end:val_end],
            "test": images[val_end:]
        }
        
        # 將圖片移動到對應的資料夾
        for split, split_images in splits.items():
            split_dir = os.path.join(output_dirs[split], class_name)
            os.makedirs(split_dir, exist_ok=True)
            for img in split_images:
                shutil.move(os.path.join(class_path, img), os.path.join(split_dir, img))
        
        # 移除原類別資料夾（已空）
        os.rmdir(class_path)

# 生成 COCO 格式的 YAML 文件
def create_coco_yaml(output_file="coco_dataset.yaml"):
    yaml_data = {
        split: os.path.abspath(path) for split, path in output_dirs.items()
    }
    
    # 收集類別名稱
    class_names = sorted(os.listdir(output_dirs["train"]))  # 使用 train 資料夾中的類別名稱
    yaml_data["names"] = {i: class_name for i, class_name in enumerate(class_names)}
    
    # 保存為 YAML 文件
    with open(output_file, "w") as yaml_file:
        yaml.dump(yaml_data, yaml_file, default_flow_style=False)
    
    print(f"COCO 格式的 YAML 文件已生成：{output_file}")

# 主程式
if __name__ == "__main__":
    create_dirs()
    split_dataset()
    create_coco_yaml()


COCO 格式的 YAML 文件已生成：coco_dataset.yaml


In [ ]:
# from ultralytics import settings

# # Update a setting
# settings.update({"runs_dir": "/path/to/runs"})

# # Update multiple settings
# settings.update({"runs_dir": "/path/to/runs", "tensorboard": False})

# # Reset settings to default values
# settings.reset()

In [ ]:
from ultralytics import YOLO

# Create a new YOLO model from scratch
# model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
# model = YOLO("yolo11n.pt")

# model = YOLO("yolo11n.yaml")  # build a new model from YAML
model = YOLO(r"runs\detect\train6\weights\best.pt")  # load a pretrained model (recommended for training)
# model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
# results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data=r"D:\team_project\yolo_v11n\tacoDataSet\data.yaml", epochs=200,imgsz=416,amp=True)

# Evaluate the model's performance on the validation set
# results = model.val()

# Perform object detection on an image using the model
# results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
# success = model.export(format="onnx")

New https://pypi.org/project/ultralytics/8.3.53 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.9.21 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 2050, 4096MiB)
engine\trainer: task=detect, mode=train, model=runs\detect\train6\weights\best.pt, data=D:\team_project\yolo_v11n\tacoDataSet\data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, reti

train: Scanning D:\team_project\yolo_v11n\tacoDataSet\train\labels.cache... 4200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4200/4200 [00:00<?, ?it/s]
val: Scanning D:\team_project\yolo_v11n\tacoDataSet\valid\labels.cache... 1704 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1704/1704 [00:00<?, ?it/s]


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to runs\detect\train7
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      1.45G      1.134      1.261      1.136         40        416: 100%|██████████| 263/263 [01:53<00:00,  2.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 54/54 [00:27<00:00,  1.95it/s]


                   all       1704       4830       0.33      0.194      0.169      0.111

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1704       4830      0.552      0.277      0.287      0.211

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     54/200      1.51G      1.084     0.9677      1.034         27        416: 100%|██████████| 263/263 [00:51<00:00,  5.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 54/54 [00:16<00:00,  3.19it/s]

                   all       1704       4830      0.549      0.291      0.305      0.225



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     55/200      1.34G      1.057     0.9395      1.019         37        416:  21%|██        | 54/263 [20:07<00:34,  6.10it/s]    

In [ ]:
# Customize validation settings
validation_results = results.val(data=r"D:\team_project\yolo_v11n\tacoDataSet\data.yaml", imgsz=224, batch=16, conf=0.25, iou=0.6, device="0")

Training setting see url:
https://docs.ultralytics.com/modes/train/#resuming-interrupted-trainings
